In [ ]:
""" Pip Install """

%pip install requests bs4

In [3]:
""" Imports """

import requests
from bs4 import BeautifulSoup
from urllib.parse import unquote, urlparse
import json

In [4]:
""" Aux func """

def print_each(list_of_links):
    """ print each """
    for link in list_of_links:
        print(link)

def print_size(list_of_links, link = ''):
    """ print size """
    print('size:', '\t', len(list_of_links), '\t', link)

def dedupe_and_sort(links):
    """ filter and sort links """
    links = list(set(links)) # deduping
    links.sort() # sorting
    return links

In [5]:
""" Scraping """

base_link = 'https://lgbtqia.fandom.com/wiki/'

initial_links = ['', 'Special:AllPages', 'Special:AllPages?from=LGBTQA',
         'Special:Categories', 'Category:Flags', 'Category:Stubs',]

initial_links = [base_link + link for link in initial_links]

allHyperLinks = {}

def clean_found_hlinks(all_a):
    """ clean found hyperlinks """
    current_hyperlinks = set()
    for link in all_a:
        got_href = link.get('href')
        if got_href is not None:
            got_href = got_href.lstrip('/')
            got_href = got_href.lower()
            full_link = got_href
            if got_href.startswith('wiki/'):
                full_link = 'https://lgbtqia.fandom.com/' + got_href
            if full_link.startswith(base_link):
                current_hyperlinks.add(full_link)
    return current_hyperlinks

def get_hyperlinks(url_soup):
    """ get hyperlinks """
    all_a = url_soup.find_all('a')
    cleaned_hlinks = clean_found_hlinks(all_a)
    return cleaned_hlinks

def get_soup(link):
    """ parse link """
    page = requests.get(link)
    if page.status_code != 200:
        return None
    soup = BeautifulSoup(page.content, 'html.parser')
    return soup

def process_all_website():
    """ process all website """
    for link in initial_links:
        soup = get_soup(link)
        hyperlinks = get_hyperlinks(soup)
        allHyperLinks[link] = {'soup': soup, 'hyperlinks': hyperlinks}

process_all_website()

print('collected links once')

collected links once


In [18]:
initial_processed_links = allHyperLinks.copy()

In [ ]:
""" Process everything
converts all links from sets into a single set to be processed.
Each link that was not already processed is processed and added to the processed links.
The processed link is removed from the unprocessed links.
If new found links are not already in the processed links, they are added to the unprocessed links set.
"""

unprocessed_links = set()

def process_all_links():
    links_to_process = set()
    for link in allHyperLinks.values():
        links_to_process.update(link['hyperlinks'])
    print(len(links_to_process))
    # print_each(sorted(list(links_to_process)))
    while links_to_process:
        link = links_to_process.pop()
        if link in allHyperLinks:
            continue
        soup = get_soup(link)
        current_hyperlinks = set() if soup is None else get_hyperlinks(soup)    
        allHyperLinks[link] = {'soup': soup, 'hyperlinks': current_hyperlinks}
        links_to_process.update(current_hyperlinks - set(allHyperLinks.keys()))
        
        print(f'processed/total: {len(allHyperLinks)}/{len(links_to_process)}', link)

process_all_links()

1013
processed/total: 58/968 https://lgbtqia.fandom.com/wiki/sexual_orientation
processed/total: 59/973 https://lgbtqia.fandom.com/wiki/asexual_spectrum#fraysexual
processed/total: 60/972 https://lgbtqia.fandom.com/wiki/cisgender?action=purge
processed/total: 61/971 https://lgbtqia.fandom.com/wiki/file:panromantic_pride_flag.png
processed/total: 62/977 https://lgbtqia.fandom.com/wiki/heteroqueer
processed/total: 63/976 https://lgbtqia.fandom.com/wiki/file:labrys_lesbian_flag.svg
processed/total: 64/979 https://lgbtqia.fandom.com/wiki/orbisian
processed/total: 65/978 https://lgbtqia.fandom.com/wiki/file:genderfluid_flag.svg
processed/total: 66/981 https://lgbtqia.fandom.com/wiki/genderf*ck
processed/total: 67/980 https://lgbtqia.fandom.com/wiki/trixic
processed/total: 68/979 https://lgbtqia.fandom.com/wiki/otter
processed/total: 69/983 https://lgbtqia.fandom.com/wiki/polyamory
processed/total: 70/994 https://lgbtqia.fandom.com/wiki/pulse_tragedy
processed/total: 71/1021 https://lgbtqia.

Exception ignored in: <finalize object at 0x2b30ab53c60; dead>
Traceback (most recent call last):
  File "b:\Programas\Programming\Python\Lib\weakref.py", line 588, in __call__
    info = self._registry.pop(self, None)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt: 


processed/total: 104/1048 https://lgbtqia.fandom.com/wiki/affixes
processed/total: 105/1047 https://lgbtqia.fandom.com/wiki/bi-lesbian
processed/total: 106/1049 https://lgbtqia.fandom.com/wiki/sapphic
processed/total: 107/1048 https://lgbtqia.fandom.com/wiki/gender_roles
processed/total: 108/1052 https://lgbtqia.fandom.com/wiki/category:articles_with_unsourced_information
processed/total: 109/1230 https://lgbtqia.fandom.com/wiki/drag?action=history
processed/total: 110/1237 https://lgbtqia.fandom.com/wiki/egogender
processed/total: 111/1293 https://lgbtqia.fandom.com/wiki/multigender?action=history
processed/total: 112/1292 https://lgbtqia.fandom.com/wiki/norm?action=purge
processed/total: 113/1291 https://lgbtqia.fandom.com/wiki/file:twink_flag.svg
processed/total: 114/1290 https://lgbtqia.fandom.com/wiki/multigender
processed/total: 115/1289 https://lgbtqia.fandom.com/wiki/file:a-spec_main_flag.svg
processed/total: 116/1288 https://lgbtqia.fandom.com/wiki/reciproromantic
processed/to

In [24]:
print(set([1, 2, 3]) - set([2, 3, 4]))

{1}


In [ ]:


def save_to_json(data, filename):
    """
    Salva o dicionário como arquivo JSON, convertendo objetos BeautifulSoup para string
    """
    def soup_to_str(obj):
        if isinstance(obj, BeautifulSoup):
            return str(obj)
        return obj

    processed_data = {}
    for key, value in data.items():
        processed_data[key] = {
            'soup': str(value['soup']),  # Converte BeautifulSoup para string
            'hyperlinks': list(value['hyperlinks'])  # Converte set para lista
        }

    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(processed_data, f, ensure_ascii=False, indent=2)

# Após process_all_links(), adicione:
save_to_json(initial_processed_links, 'scraped_data.json')

In [ ]:
""" Check items that are in all lists """

def get_common_links(all_links):
    """ get common links """
    common_links = set(all_links[0])
    for links in all_links[1:]:
        common_links.intersection_update(links)
    return common_links

common_links = get_common_links(allHyperLinks)

print_size(common_links, 'common links')

print(common_links)

In [ ]:
""" Filtering Links """

print('size:', len(allHyperLinks))
allHyperLinks = dedupe_and_sort(allHyperLinks)
print('size:', len(allHyperLinks))


In [ ]:
""" Second Iteration """

allHyperLinks += get_hyperlinks_from_list_of_links(allHyperLinks)


In [ ]:
print('size:', len(allHyperLinks))
filteredAllHyperLinks = dedupe_and_sort(allHyperLinks).copy()
print('size:', len(allHyperLinks))

In [ ]:
""" More filtering """

def link_404(link):
    """ remove 404 links """
    page = requests.get(link)
    return page.status_code == 404

def link_question(link):
    """ remove question links """
    return '?' in link

def link_hashtag(link):
    """ remove hashtag links """
    return '#' in link

def filtering_links(links):
    """ filtering links """
    for link in links:
        print(link)
        if link_question(link) or link_404(link) or link_hashtag(link):
            print('removing:', link)
            links.remove(link)
    return links



print_size(filteredAllHyperLinks)
# filteredAllHyperLinks = filtering_links(filteredAllHyperLinks)
print_size(filteredAllHyperLinks)

In [ ]:
unexisting_pages = [
    'https://lgbtqia.fandom.com/wiki/2-spirit',
    'https://lgbtqia.fandom.com/wiki/ace_lesbian',
    'https://lgbtqia.fandom.com/wiki/equality_act',
    'https://lgbtqia.fandom.com/wiki/adonic',
    'https://lgbtqia.fandom.com/wiki/equality_act_2010',
]

for link in unexisting_pages:
    page = requests.get(link)
    print(page.status_code)
    